In [ ]:
import os
import re
import time
import random
import math
import requests
import json
import datetime
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup

In [60]:
# 读取需要的环境变量
load_dotenv()
# link info
SF_URL = os.getenv('SF_URL')
LDTJ_URL = os.getenv('LDTJ_URL')
JBXX_URL = os.getenv('JBXX_URL')
CTD_URL = os.getenv('CTD_URL')
IMG_URL = os.getenv('IMG_URL')
DSH_URL = os.getenv('DSH_URL')
YSH_URL = os.getenv('YSH_URL')
DSH_PAGE = os.getenv('DSH_PAGE')
YSH_PAGE = os.getenv('YSH_PAGE')
PAGE_TAIL = os.getenv('PAGE_TAIL')
BASE_NAME = os.getenv('BASE_NAME')
# headers info 
img_headers = {
    "Accept": os.getenv("ACCEPT"),
    "Accept-Encoding": os.getenv("ACCEPT_ENCODING"),
    "Accept-Language": os.getenv("ACCEPT_LANGUAGE"),
    "Authorization": os.getenv("AUTHORIZATION"),
    "Connection": os.getenv("CONNECTION"),
    "Content-Length": os.getenv("CONTENT_LENGTH"),
    "Content-Type": os.getenv("CONTENT_TYPE"),
    "Cookie": os.getenv("COOKIE"),
    "Host": os.getenv("HOST"),
    "origin": os.getenv("ORIGIN"),
    "Referer": os.getenv("REFERER"),
    "Sec-Ch-Ua": os.getenv("SEC_CH_UA"),
    "Sec-Ch-Ua-Mobile": os.getenv("SEC_CH_UA_MOBILE"),
    "Sec-Ch-Ua-Platform": os.getenv("SEC_CH_UA_PLATFORM"),
    "Sec-Fetch-Dest": os.getenv("SEC_FETCH_DEST"),
    "Sec-Fetch-Mode": os.getenv("SEC_FETCH_MODE"),
    "Sec-Fetch-Site": os.getenv("SEC_FETCH_SITE"),
    "User-Agent": os.getenv("USER_AGENT")
}
normal_headers = {
    "Accept": os.getenv("ACCEPT"),
    "Accept-Encoding": os.getenv("ACCEPT_ENCODING"),
    "Accept-Language": os.getenv("ACCEPT_LANGUAGE"),
    "Authorization": os.getenv("AUTHORIZATION"),
    "Connection": os.getenv("CONNECTION"),
    "Cookie": os.getenv("COOKIE"),
    "Host": os.getenv("HOST"),
    "Referer": os.getenv("REFERER"),
    "Sec-Ch-Ua": os.getenv("SEC_CH_UA"),
    "Sec-Ch-Ua-Mobile": os.getenv("SEC_CH_UA_MOBILE"),
    "Sec-Ch-Ua-Platform": os.getenv("SEC_CH_UA_PLATFORM"),
    "Sec-Fetch-Dest": os.getenv("SEC_FETCH_DEST"),
    "Sec-Fetch-Mode": os.getenv("SEC_FETCH_MODE"),
    "Sec-Fetch-Site": os.getenv("SEC_FETCH_SITE"),
    "User-Agent": os.getenv("USER_AGENT")}


In [63]:
### 辅助函数
# 装饰器，如果函数没有返回值，则一直重试
def retry_if_no_return(func):
    def wrapper(*args, **kwargs):
        # 定义重试次数
        retry_times = 5
        while retry_times>=0:
            result = func(*args, **kwargs)
            if result:
                return result
            # 如果函数没有返回值，可以在这里添加日志或等待时间
            time.sleep(2)
            retry_times -= 1
    return wrapper

# 遍历url列表下载图片
@retry_if_no_return
def download_image(url,save_path_name):
    headers = {'User-Agent':os.getenv('USER_AGENT')}
    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 200:
            with open(save_path_name, 'wb') as f:
                f.write(r.content)
                return True
    except:
        return False
    time.sleep(random.random())

In [ ]:
# 获取施肥信息
def get_sf_info(point_id,headers):
    """获取施肥信息"""
    url = f"{SF_URL}{point_id}"
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取图片信息
def get_img_info(point_id,headers):
    """获取图片和视频链接"""
    url = IMG_URL
    # 请求的JSON数据
    # 请根据实际情况替换下面的data字典
    data = {
        "glbh": f"{int(point_id)}"
    }
    # 将字典转换为JSON格式的字符串
    payload = json.dumps(data)
    # 发送POST请求
    response = requests.post(url, headers=headers, data=payload)

    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取立地条件信息
def get_ldtj_info(point_id,headers):
    """获取立地条件信息"""
    url = f"{LDTJ_URL}{point_id}"
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取样点基本信息
def get_base_info(point_id,headers):
    """获取基本信息"""
    url = f"{JBXX_URL}{point_id}"
        # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取采土袋信息
def get_ctd_info(point_id,headers):
    """获取采土袋信息"""
    url = f"{CTD_URL}{point_id}"
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取当前页面cookie信息
def get_cookie():
    """获取当前页面cookie信息"""
    # 使用JavaScript获取Cookie
    cookie_script = """
    return document.cookie;
    """
    cookie_value = driver.execute_script(cookie_script)
    return cookie_value
# 更新cookie信息
def update_cookie(headers):
    """更新cookie信息"""
    # 获取当前页面cookie信息
    cookie_value = get_cookie()
    # 更新headers
    headers.update({"Cookie": cookie_value})
    return headers
# 获取已审核数量
def get_audit_num(headers):
    """获取已审核数量"""
    url = YSH_URL
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data['result']
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取待审核数量
def get_wait_num(headers):
    """获取待审核数量"""
    url = DSH_URL
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data['result']
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取指定页面的编码
def get_set_page_num(url,headers):
    """获取指定页面的编码"""
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        info_list = data['result']['records']
        # 循环获取编码
        temp_list = [_['ydbh'] for _ in info_list]
        return temp_list
    else:
        print("请求失败，状态码：", response.status_code)
        return False

# 获取已审核的编码
def get_page_number(total,headers):
    """获取当前传入页面的编码,默认每一页50"""
    # 计算有多少页
    page_num = math.ceil(total / 50)
    # 循环获取每一页的编码
    result_list = []
    for i in range(1, page_num + 1):
        url = f"{YSH_PAGE}{i}{PAGE_TAIL}"
        result_list+=get_set_page_num(url,headers)
        time.sleep(random.random())
    return result_list

# 获取待审核的编码
def get_wait_page_number(total,headers):
    """获取当前传入页面的编码,默认每一页50"""
    # 计算有多少页
    page_num = math.ceil(total / 50)
    # 循环获取每一页的编码
    temp_list = []
    for i in range(1, page_num + 1):
        url = f"{DSH_PAGE}{i}{PAGE_TAIL}"
        temp_list+=get_set_page_num(url,headers)
        time.sleep(random.random())
    return temp_list
    

In [ ]:
# 配置浏览器
# chrome.exe --remote-debugging-port=9999 --user-data-dir="D:\Selenium\AutomationProfile"
# chrome.exe --remote-debugging-port=9999 --user-data-dir="D:\Program Files\ChromeDir"
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "localhost:9998")
driver = webdriver.Chrome(options=options)

driver.implicitly_wait(60)

In [ ]:
# 保存路径
save_path = r"F:\collection_spb_info\李相楹"

# 检查heraders并更新

In [ ]:

headers = update_cookie(normal_headers)

In [ ]:
wait_point_list = get_wait_page_number(get_wait_num(headers),headers)

In [ ]:
# 获取当前账户下的待审核和已审核编码，并写入文件
# 更新headers
headers = update_cookie(headers)
audit_point_list = get_page_number(get_audit_num(headers),headers)
wait_point_list = get_wait_page_number(get_wait_num(headers),headers)
all_point = audit_point_list + wait_point_list

In [ ]:
# 写入编码到文件
with open(os.path.join(save_path,'point_id.txt'),'w',encoding='utf-8') as f:
    for one_id in all_point:
        f.write(one_id+'\n')

In [ ]:
len(all_point),len(wait_point_list),len(audit_point_list)

In [ ]:
# 测试一个点位的基本信息
base_info = get_base_info(all_point[0],headers)
base_info

In [ ]:
# 获取基本信息
# 更新headers
headers = update_cookie(headers)
base_info_list = []
for one_point in tqdm(all_point):
    one_point_base_info = get_base_info(one_point,headers)['result']
    # 随机休眠
    time.sleep(random.random())
    base_info_list.append(one_point_base_info)


In [ ]:
df_base_info = pd.DataFrame(base_info_list)
df_base_info

In [ ]:
# 保存基本信息
df_base_info.to_excel(os.path.join(save_path,'base_info.xlsx'),index=False)

In [ ]:
# 测试一个点位的图片信息
img_info = get_img_info(all_point[0],img_headers)
img_info

In [ ]:
# 获取图片链接信息
# 更新headers
img_headers = update_cookie(img_headers)
img_info_list = []
for one_point in tqdm(all_point):
    one_point_img_info = get_img_info(one_point,img_headers)['result']
    # 为链接增加域名
    prefix = BASE_NAME
    updated_list = [{**item, 'wjlj': f"{prefix}{item['wjlj']}"} for item in one_point_img_info]
    # 随机休眠
    time.sleep(random.random())
    img_info_list+=updated_list

In [ ]:
df_img_info = pd.DataFrame(img_info_list)
df_img_info

In [ ]:
# 保存媒体信息
df_img_info.to_excel(os.path.join(save_path, 'img_info.xlsx'), index=False)

In [ ]:
# 测试一个点位的立地调查信息
ldtj_info = get_ldtj_info(all_point[0],headers)
ldtj_info

In [ ]:
# 获取立地条件信息
ldtj_info_list = []
for one_point in tqdm(all_point):
    one_point_ldtj_info = get_ldtj_info(one_point,headers)['result']
    ldtj_info_list.append(one_point_ldtj_info)
    # 随机休眠
    # time.sleep(random.random())

In [ ]:
df_ldtj_info = pd.DataFrame(ldtj_info_list)
df_ldtj_info

In [ ]:
# 保存立地条件信息
df_ldtj_info.to_excel(os.path.join(save_path,'ldtj_info.xlsx'),index=False)

In [ ]:
# 测试一个点位的采土袋信息
ctd_info = get_ctd_info(all_point[0],headers)
ctd_info

In [ ]:
# 获取采土袋信息
ctd_info_list = []
for one_point in tqdm(all_point):
    one_point_ctd_info = get_ctd_info(one_point,headers)['result']
    ctd_info_list+=one_point_ctd_info
    # 随机休眠
    # time.sleep(random.random())

In [ ]:
df_ctd_info = pd.DataFrame(ctd_info_list)
df_ctd_info

In [ ]:
# 保存采土袋信息
df_ctd_info.to_excel(os.path.join(save_path, 'ctd_info.xlsx'), index=False)

In [ ]:
# 测试一个点位的施肥信息
sf_info = get_sf_info(all_point[0],headers)
sf_info

In [ ]:
# 获取施肥信息
sf_info_list = []
for one_point in tqdm(all_point):
    one_point_sf_info = get_sf_info(one_point,headers)['result']
    sf_info_list+=one_point_sf_info
    # 随机休眠
    # time.sleep(random.random())

In [ ]:
df_sf_info = pd.DataFrame(sf_info_list)
df_sf_info

In [ ]:
# 保存施肥信息
df_sf_info.to_excel(os.path.join(save_path, 'sf_info.xlsx'), index=False)